In [2]:
import modal
from dotenv import load_dotenv


app = modal.App(name="deals-finding-agent")
Image = modal.Image.debian_slim(python_version="3.12")

In [3]:
def compress_dictionary(dictionary):
    return "".join([f"{key}: {value}" for key, value in dictionary.items()])

@app.function(image=Image)
def comp():
    dictionary = {"hello": "world", "foo": "bar"}
    compressed = compress_dictionary(dictionary)
    print("Compressed dictionary: ", compressed)
    return compressed

@app.local_entrypoint()
def main():
    print(comp.remote())  # Modal functions use .remote() or .local(), not ()

In [6]:
with app.run():
    val = comp.remote()
    print(val)

hello: worldfoo: bar


In [4]:
# Test Modal connection: run a remote function and verify the result
def test_modal_connection():
    with app.run():
        result = comp.remote()
        expected = "hello: worldfoo: bar"
        assert result == expected, f"Expected {expected!r}, got {result!r}"
    print("Modal connection test passed.")

test_modal_connection()

Modal connection test passed.


## Testing the Price Prediction finetuned model

In [6]:
from pricer_service import app, Pricer

In [5]:
with app.run():
    pricer = Pricer()
    result = pricer.price.remote("LG washing machine with topload dryer and washer");
result

NameError: name 'result' is not defined

### Deploy this Modal Setup Function to  Infra

In [7]:
!uv run modal deploy -m pricer_service

- Creating objects...
| Creating objects...
└── - Creating mount c:\Users\psiva\Documents\AI 
    Projects\Deals_Finding_Agent\pricer_service.py: Uploaded 0/1 files
- Creating objects...
└── | Creating mount c:\Users\psiva\Documents\AI 
    Projects\Deals_Finding_Agent\pricer_service.py: Finalizing index of 1 files
| Creating objects...
├── 🔨 Created mount c:\Users\psiva\Documents\AI 
│   Projects\Deals_Finding_Agent\pricer_service.py
└── 🔨 Created function Pricer.*.
| Creating objects...
├── 🔨 Created mount c:\Users\psiva\Documents\AI 
│   Projects\Deals_Finding_Agent\pricer_service.py
└── 🔨 Created function Pricer.*.

✓ Created objects.
├── 🔨 Created mount c:\Users\psiva\Documents\AI 
│   Projects\Deals_Finding_Agent\pricer_service.py
└── 🔨 Created function Pricer.*.
✓ App deployed in 1.513s! 🎉

View Deployment: 
https://modal.com/apps/p-sivakrishnareddy-siva/main/deployed/pricer-service


## RAG (Retrieval Augmented Generation) based on a dataset of 800,000 scraped Amazon products

#### For our 2nd agent, we will use OpenAI to estimate the price of one of our deals - with additional setup

we can beat a frontier LLM by fine-tuning an open-source LLM for price estimation.

Now we are going to try **inference time** techniques instead of training -- by using RAG!

In [8]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item